In [1]:
import geopandas as gpd
import pandas as pd
from utils import gdf_operations, functions

2025-01-21 15:56:14.414 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [2]:
distrito =gdf_operations.get_dados('distrito')
subbac = gdf_operations.get_dados('subbac')
subpref = gdf_operations.get_dados('subpref')
fcu = gdf_operations.get_dados('fcu')

In [3]:
unidades_list = [
    ("Sub Bacias Hidrográficas", "Lorem ipsum dolor sit amet...", 'subbac', 'nm_bacia_h'),
    ("Subprefeituras", "Lorem ipsum dolor sit amet...", 'subpref', 'nm_subpref'),
    ("Distritos", "Lorem ipsum dolor sit amet...", 'distrito', 'nm_distrit'),
    ("Favelas e Comunidades Urbanas", "Lorem ipsum dolor sit amet...", 'fcu', 'nm_fcu')
    
]
unidades_df = pd.DataFrame(unidades_list, columns=['name', 'desc', 'gdf_name', 'column_name'])

Vou começar pelo fcu para ver se dá certo.

In [4]:
def overlay_intersec(unidades_df, 
                     choice_unidade, 
                     gdf_unidade,
                     gdf_intersec,
                    ):
    
    name_gdf_intersec = functions.find_gdf_info(unidades_df, gdf_intersec, 'gdf_name', 'name')
    gdf_intersec = gdf_operations.create_gdf_sorted(
        gdf_intersec, 
        name_gdf_intersec,
        isIntersec=True
        )
    

    overlay_unidade = gpd.overlay(
        gdf_unidade,
        gdf_intersec,
        how= 'intersection',
        keep_geom_type=True
    )
    
    overlay_unidade = (
        overlay_unidade.explode(
            index_parts=False
        )
    )
    overlay_unidade['area'] = overlay_unidade['geometry'].area

    overlay_unidade = (
        overlay_unidade.sort_values(
            by='area', 
            ascending=False
        )
    )

    return overlay_unidade


# FCU

In [5]:
fcu_sorted = gdf_operations.create_gdf_sorted(
    fcu, 'fcu'
)

C:\Users\x526378\Desktop\projetos\saneamento\hydra\streamlit-pmsb\utils\gdf_operations.py:69: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted[column] = gdf[column]


In [6]:
choice_unidade = 'fcu'
gdf_unidade = fcu_sorted
name_column_unidade = functions.find_gdf_info(unidades_df, choice_unidade, 'gdf_name', 'column_name')

In [7]:
 gdf_intersec_a = gdf_operations.create_gdf_sorted(
        distrito, 
        'distrito',
        isIntersec=True
        )
#gdf_intersec_a.head()

C:\Users\x526378\Desktop\projetos\saneamento\hydra\streamlit-pmsb\utils\gdf_operations.py:58: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted['geometry'] = gdf['geometry']


In [8]:
overlay_fcu_distrito = gpd.overlay(
    gdf_unidade,
    gdf_intersec_a,
    how='intersection',
    keep_geom_type=True
)

overlay_fcu_distrito = (
        overlay_fcu_distrito.explode(
            index_parts=False
        )
)
overlay_fcu_distrito['area'] = overlay_fcu_distrito['geometry'].area

overlay_fcu_distrito = (
    overlay_fcu_distrito.sort_values(by='area', 
            ascending=False)
)

In [9]:
unique_unidades_fcu_dist = (
        overlay_fcu_distrito[
            'cd_fcu' #trocar pelo cd
        ].unique().tolist()
    )
unique_unidades = unique_unidades_fcu_dist
overlay_unidade=overlay_fcu_distrito
name_column_unidade = 'cd_fcu'
name_column_intersec = 'cd_distrit'

columns_overlay = overlay_unidade.columns
gdf_final_unidade = gpd.GeoDataFrame(columns=columns_overlay)

row_idx=0

for u in unique_unidades:
    gdf_unique = overlay_unidade[overlay_unidade[name_column_unidade] == u].head(1)
    
    # Adiciona as linhas filtradas ao DataFrame final
    for _, linha in gdf_unique.iterrows():
        gdf_final_unidade.loc[row_idx] = linha
        row_idx += 1

    
    

       
